In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import re
import sys
import time
import pprint
import calendar
import datetime
import pandas as pd
from requests_html import HTMLSession
from IPython.display import clear_output

In [2]:
#保存フォルダ指定
path = r"Z:\Share\dl\day"

In [3]:
def loadData():
    #発売日
    year = int(dl_year) #int(match0])
    month = int(dl_month) #int(match[1])
    day =  int(dl_day) #int(match[2])
    date = str(year) + '/' + str(month) + '/' + str(day) 
    print(date)

    #曜日算出
    weekday = datetime.datetime(year, month, day).strftime('%a')
#     print(weekday)

    #その日の発売本数
    sel = '#box1 > div > h3 > span'
    result = r.html.find(sel, first=True).text
    # print(result)

    #数字だけ抽出
    match = re.compile('\d+').findall(result)
    num_of_sale = int(match[0])
#     print(num_of_sale)
    return date,weekday,num_of_sale

In [4]:
def loadDetail():
    str_index = str(i + 1)
    sel = '#box1 > div > div.n_worklist.type_2col > div:nth-child(' + str_index + ')'
    result = r.html.find(sel, first = True)

    #ジャンル
    Category = result.find('div.work_thumb')[0].text
#         print(Category)

    #タイトル
    Title = result.find('dl > dt > a')[0].text
    print(Title)

    #URL(set型になってるので一旦リストにしてstr型に変換)
    url = result.find('dl > dt > a')[0].links
    url = list(url)[0]
#         print(url)

    #サークル
    circle = result.find('dl > dd.maker_name')[0].text
#         print(circle)

    #価格
    price = result.find('dl > dd.work_price_wrap > span.work_price')[0].text
    price = int(re.sub("\\D", "", price))
#         print(price)

    #割引
    try:
        discount = result.find('dl > dd.work_price_wrap > span.work_point')[0].text
        discount = discount.split()
        discount_yen = int(re.sub("\\D", "", discount[0]))
        discount_percent = int(re.sub("\\D", "", discount[1]))
#         print(discount)
#         print(discount_yen,discount_percent)
    except IndexError as e:
        discount_yen = 0
        discount_percent = 0

    #専売
    monopoly = result.find('dl > dt > div > span')
    if not monopoly:
        monopoly = '非専売'
#             print('NonMonopoly')
    elif result.find('dl > dt > div > span')[0].text == 'DLsite専売':
        monopoly = '専売'
#         monopoly = result.find('dl > dt > div > span')[0].text
#             print(monopoly)
    else:
        monopoly = 0

    #説明文
    explanation = result.find('dl > dd.work_text')[0].text
    explanation = explanation.replace('\n', '')
#         print(explanation)

    #性癖
    hentai_flag = False
    hentai_int = 1
    hentai_list = list()
    while hentai_flag == False:
        try:
            hentai = result.find('dl > dd.search_tag > a:nth-child(' + str(hentai_int) + ')')[0].text
            hentai_list.append(hentai)
            hentai_int+=1
        except IndexError as e:
            hentai_flag = True
#         print(hentai_list)
    hentai_join = '_'.join(hentai_list)

    #販売数
    try:
        sale = result.find('dl > dd.work_sales_info > div.work_dl > span')[0].text
        sale = int(re.sub("\\D", "", sale))
#             print(sale)
    except IndexError as e:
        sale = 0
#             print(sale)

    #評価数
    try:
        evaluation = result.find('dl > dd.work_sales_info >div')[1].text
        evaluation = int(re.sub("\\D", "", evaluation))
#             print(evaluation)
    except IndexError as e:
        evaluation = 0
#             print(evaluation)

    #スター
    try:
        star = r.html.find('#box1 > div > div.n_worklist.type_2col > div:nth-child(' + str_index + ') > dl > dd.work_sales_info > div', first = False)[1]
        star = re.compile('\d+').findall(star.attrs['class'][1])
        star = int(star[0])
#             print(star)
    except IndexError as e:
        star = 0
#             print(star)

    #レビュー
    try:
        review = result.find('dl > dd.work_sales_info > div.work_review > div')[0].text
        review = int(re.sub("\\D", "", review))
    except IndexError as e:
        review = 0
#     print(review)

    print(Category,Title,url,circle,price,discount_yen,discount_percent,monopoly,explanation,hentai_join,sale,evaluation,star,review)
    return Category,Title,url,circle,price,discount_yen,discount_percent,monopoly,explanation,hentai_join,sale,evaluation,star,review

In [5]:
%%time

df = pd.DataFrame(columns=['発売日','曜日','ジャンル','タイトル','URL','サークル','価格','割引[円]','割引%',\
                           '専売','説明文','性癖','販売数','評価数','スター','レビュー'] )
df_month = df
df_year = df
df_all = df

session = HTMLSession()

start_year = 2000
start_day = 1
dl_year = 2020

for dl_year in range(start_year,dl_year + 1,1):
    
    for dl_month in range(1, 12 + 1, 1):
#         デバッグ用
#         if dl_month == 2:
#             break

        #その月の最終日を取得
        day_range = calendar.monthrange(int(dl_year), dl_month)[1]

        for dl_day in range(start_day, day_range + 1, 1):
#             デバッグ用
#             if int(dl_day) == 2:
#                 sys.exit()



            #DataFrame初期化
            df = df[:0]

            #セッションの開始
            dl_year = str(dl_year)
            dl_month = str(dl_month).zfill(2)
            dl_day = str(dl_day).zfill(2)
            time.sleep(5)
            r = session.get('https://www.dlsite.com/maniax/new/=/date/' + dl_year + '-' + dl_month + '-' + dl_day + '/cdate/' + dl_year + '-' + dl_month + '/show_layout/2')

            #結果クリア
            clear_output()
            
            #発売日、曜日、発売本数取得
            date,weekday,num_of_sale = loadData()

            #作品詳細
            for i in range(num_of_sale):
                #デバッグ用
    #             if i == 2:
    #                 break

                #作品詳細取得
                Category,Title,url,circle,price,discount_yen,discount_percent,\
                monopoly,explanation,hentai_join,sale,evaluation,star,review = loadDetail()

                #データフレームに追加
                tmp_se = pd.Series( [date,weekday,\
                                     Category,Title,url,circle,price,discount_yen,discount_percent,\
                                     monopoly,explanation,hentai_join,sale,evaluation,star,review],\
                                   index=df.columns)
                df = df.append(tmp_se,ignore_index=True)
#                 break

            #日毎にエクスポート
            df.to_csv(path + '\dl_' + dl_year + '-' + dl_month + '-' + dl_day + '.csv')
            # df_month = df_month.append(df,ignore_index=True)
        
#         #月毎にエクスポート
        # df_month.to_csv(path + '\dl_' + dl_year + '-' + dl_month + '.csv')
        # df_year = df_year.append(df_month,ignore_index=True)
        
#     #年毎にエクスポート
    # df_year.to_csv(path + '\dl_' + dl_year + '.csv')
#     df_all = df_all.append(df_year,ignore_index=True)

#全部まとめてエクスポート
# df_all.to_csv(path + '\dl_all' + '.csv')

2020/12/31
メスガキちゃんミニ音声ドラマ(仮)
ボイス・ASMR メスガキちゃんミニ音声ドラマ(仮) https://www.dlsite.com/maniax/work/=/product_id/RJ294391.html マザースター / 一之瀬りと 77 7 10 専売 童貞にやさしいメスガキちゃんといちゃいちゃして癒される音声作品です。メスガキ癒し音声です。 癒し_バイノーラル/ダミヘ_ASMR_ラブラブ/あまあま_言葉責め_ささやき_ロリ_先輩/後輩 60 5 50 2
NARITA Vol.3
CG・イラスト NARITA Vol.3 https://www.dlsite.com/maniax/work/=/product_id/RJ294008.html NAZE式 1100 100 10 非専売 超乳!陥没乳首!乳首責め!母乳!ひたすらおっぱいだけでイキ狂うCG集第三弾です。 おっぱい_母乳_搾乳_ニプルファック 52 9 50 1
スワッピングビートDX
マンガ スワッピングビートDX https://www.dlsite.com/maniax/work/=/product_id/RJ294724.html ヴェロニカの歯 660 60 10 非専売 超○ヒロインカップル同士の背徳の恋人交換スワッピング。DL版描き下ろしエロシーン3シーン追加(差分込み11枚)。 浮気_寝取られ_寝取り_顔射_フェラチオ_複数プレイ/乱交_変身ヒロイン 36 0 0 0
家出女子校生と童貞筆下ろし恋人プレイ-お泊まりのお礼にエッチしちゃう夜-
ボイス・ASMR 家出女子校生と童貞筆下ろし恋人プレイ-お泊まりのお礼にエッチしちゃう夜- https://www.dlsite.com/maniax/work/=/product_id/RJ293464.html クッキーボイス / 大山チロル 990 90 10 専売 年下の家出娘にリードされてキスや全身スリスリのあまあま恋人プレイのちに筆下ろしを体験する音声です。CV大山チロル 約80分 ラブラブ/あまあま_手コキ_中出し_パイズリ_フェラチオ_ノーマルプレイ_学生_ビッチ 34 0 0 0
【マゾ向け男受け】オタクくんを生オナホにしてあげるね～ふたなり委員長のマゾ穴奴隷調教～【バイノーラル】
ボイス・ASMR

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   発売日     0 non-null      object
 1   曜日      0 non-null      object
 2   ジャンル    0 non-null      object
 3   タイトル    0 non-null      object
 4   URL     0 non-null      object
 5   サークル    0 non-null      object
 6   価格      0 non-null      object
 7   割引[円]   0 non-null      object
 8   割引%     0 non-null      object
 9   専売      0 non-null      object
 10  説明文     0 non-null      object
 11  性癖      0 non-null      object
 12  販売数     0 non-null      object
 13  評価数     0 non-null      object
 14  スター     0 non-null      object
 15  レビュー    0 non-null      object
dtypes: object(16)
memory usage: 0.0+ bytes
